# Пример реализации технологии blockchain
## 1 Опишем базовые классы

In [0]:
from dataclasses import dataclass
from typing import Tuple
import pandas as pd


@dataclass
class Block:
    transaction: str
    hash_code: int
        
    def calc_transaction_hash(self) -> int:
        hash_sum = 0
        
        for char in self.transaction:
            hash_sum += ord(char)
            
        return hash_sum % 42
    
    def form_block_hash(self, prev_hash: int):
        # (a + b) mod c = (a mod c + b mod c) mod c
        self.hash_code = (self.calc_transaction_hash() + prev_hash % 42) % 42
    
    
class Blockchain:
    def __init__(self):
        self.blocks = [Block('INIT', 14)]
        
    def add_transaction(self, transaction: str):
        new_block = Block(transaction, 0)
        
        new_block.form_block_hash(self.blocks[-1].hash_code)
        
        self.blocks.append(new_block)
        
    def merge_with(self, other: Blockchain) -> Tuple[bool, str]:
        # chech other chains history
        prev_hash = 0
        for my_block, other_block in zip(self.blocks, other.blocks):
            my_block.form_block_hash(prev_hash)
            other_block.form_block_hash(prev_hash)
            
            if my_block.hash_code != other_block.hash_code:
                return (False, 'Incorrect history')
            
            prev_hash = my_block.hash_code
            
        # find difference
        difference = other.blocks[len(self.blocks):]
        
        # check difference
        prev_hash_code = other.blocks[len(self.blocks) - 1].hash_code
        for block in difference:
            current_hash = (\
                prev_hash_code % 42\
                + block.calc_transaction_hash() % 42) % 42
            
            if block.hash_code != current_hash:
                return (False, 'Incorrect differencies')
            
            prev_hash_code = block.hash_code
        
        self.blocks.extend(difference)
        
        return (True, 'Merge is ok')
        
    def history(self) -> pd.DataFrame:
        history = {'transaction': [], 'hash': []}
        
        for block in self.blocks:
            history['transaction'].append(block.transaction)
            history['hash'].append(block.hash_code)
            
            
        return pd.DataFrame(history)

## 2 Сформируем 2 репликации цепочки блоков, продемонстрируем результаты

In [0]:
alice_replication = Blockchain()
bob_replication = Blockchain()


# 1 transaction
alice_replication.add_transaction('5 coins has been given to bob from system')
bob_replication.add_transaction('5 coins has been given to bob from system')

# 2 transaction
alice_replication.add_transaction('15 coins has been given to alice from system')
bob_replication.add_transaction('15 coins has been given to alice from system')

# 3 transaction
alice_replication.add_transaction('alice has given bob 5 coins')
bob_replication.add_transaction('alice has given bob 5 coins')

In [0]:
alice_replication.history()

,transaction,hash
0,INIT,14
1,5 coins has been given to bob from system,35
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1


In [0]:
bob_replication.history()

,transaction,hash
0,INIT,14
1,5 coins has been given to bob from system,35
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1


## 3 Продемонстируем локальные изменения и слияние репликаций

In [0]:
bob_replication.add_transaction('bob has returned alice 5 coins')

In [0]:
alice_replication.history()

,transaction,hash
0,INIT,14
1,5 coins has been given to bob from system,35
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1


In [0]:
bob_replication.history()

,transaction,hash
0,INIT,14
1,5 coins has been given to bob from system,35
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1
4,bob has returned alice 5 coins,30


In [0]:
alice_replication.merge_with(bob_replication)

(True, 'Merge is ok')

In [0]:
alice_replication.history()

,transaction,hash
0,INIT,14
1,5 coins has been given to bob from system,35
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1
4,bob has returned alice 5 coins,30


In [0]:
bob_replication.history()

,transaction,hash
0,INIT,14
1,5 coins has been given to bob from system,35
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1
4,bob has returned alice 5 coins,30


## 4. Продемонстрируем защиту от недопустимых изменений

In [0]:
bob_replication.blocks[1].transaction = '9999 coins has been given to bob from system'
bob_replication.blocks[1].form_block_hash(0)

In [0]:
alice_replication.history()

,transaction,hash
0,INIT,14
1,5 coins has been given to bob from system,35
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1
4,bob has returned alice 5 coins,30


In [0]:
bob_replication.history()

,transaction,hash
0,INIT,14
1,9999 coins has been given to bob from system,28
2,15 coins has been given to alice from system,14
3,alice has given bob 5 coins,1
4,bob has returned alice 5 coins,30


In [0]:
alice_replication.merge_with(bob_replication)

(False, 'Incorrect history')